# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [ ]:
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage
import os

# 你的 Key Vault 名称，例如 'my-keyvault'
key_vault_name = "kgkv4Azure4cn3"
key_vault_url = f"https://{key_vault_name}.vault.azure.cn/"

# 创建凭证和 Key Vault 客户端
credential = AzureCliCredential()
client = SecretClient(vault_url=key_vault_url, credential=credential)



# 获取 secret，secret_name 是你在 Key Vault 中保存 API key 的名字
secret_name = "openai-swedencentral"
retrieved_secret = client.get_secret(secret_name)

# 设置为环境变量或直接传入 LLM 实例
#os.environ["AZURE_OPENAI_API_KEY"] = retrieved_secret.value
api_key = retrieved_secret.value

In [ ]:
from azure_key_vault import get_kv_secret
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI




openai_endpoint_swedencentral = "https://kg-openai-swedencentral.openai.azure.com/"
#openai_key_swedencentral = os.environ["AZURE_OPENAI_API_KEY"] #get_kv_secret("openai-swedencentral")

llm = AzureChatOpenAI(
    api_version="2024-12-01-preview",
    api_key = api_key,
    azure_endpoint=openai_endpoint_swedencentral,
    azure_deployment="gpt-4o"

)

In [ ]:
import os 
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4-1106",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

## Load the dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [1]:
df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

NameError: name 'pd' is not defined

## Prepare the Langchain dataframe agent

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True)

agent.invoke("how many rows are there?")

## Design your prompt and ask your question

In [ ]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2020" 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)